In [12]:
import numpy as np
import pandas as pd
import cv2
from os import listdir
from os.path import isfile, join
import matplotlib.pyplot as plt
import torch
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader


In [45]:
import torch

print("CUDA Available:", torch.cuda.is_available())

if torch.cuda.is_available():
    print("Device Name:", torch.cuda.get_device_name(0))
else:
    print("CUDA is NOT available. Check your driver or setup.")


CUDA Available: False
CUDA is NOT available. Check your driver or setup.


In [42]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)


Using device: cpu


In [2]:
import torch

print("Torch version:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())
print("CUDA device count:", torch.cuda.device_count())

if torch.cuda.is_available():
    print("Current device:", torch.cuda.current_device())
    print("Device name:", torch.cuda.get_device_name(0))
else:
    print("❌ CUDA is NOT available. Likely driver or install issue.")


Torch version: 2.6.0+cu118
CUDA available: True
CUDA device count: 1
Current device: 0
Device name: NVIDIA GeForce GTX 1650


In [4]:
images_dest = "inaturalist_12K/train/Amphibia"

In [5]:
images = listdir(images_dest)
data = []
#extracting the images from the folder
for image in images:
    if isfile(join(images_dest, image)):
        # print(image)
        #resizing the image to 224x224
        img = cv2.imread(join(images_dest, image))
        img = cv2.resize(img, (224, 224))
        data.append(img)

In [6]:
batch_size = 32
num_classes = 10
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cuda


In [7]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],   # ImageNet mean
                         [0.229, 0.224, 0.225])   # ImageNet std
])

In [27]:
import torch
import torch.nn as nn
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
import os

# === CONFIG ===
data_dir = "./inaturalist_12K"  # Your dataset directory (train & val inside this)
batch_size = 32
num_classes = 10
num_epochs = 5
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# === TRANSFORMS ===
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],   # ImageNet mean
                         [0.229, 0.224, 0.225])   # ImageNet std
])

# === DATASET LOADING ===
train_dataset = datasets.ImageFolder(os.path.join(data_dir, "train"), transform=transform)
val_dataset = datasets.ImageFolder(os.path.join(data_dir, "val"), transform=transform)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=2)

# === LOAD GOOGLENET & MODIFY ===
model = models.googlenet(pretrained=True)

for param in model.parameters():
    param.requires_grad = False

# Replace only the final classifier layer
model.fc = nn.Linear(model.fc.in_features, num_classes)

# Enable training only on the new final layer
for param in model.fc.parameters():
    param.requires_grad = True

model.to(device)

# === LOSS & OPTIMIZER ===
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.fc.parameters(), lr=0.001)



Using device: cuda


In [28]:
# === TRAIN LOOP ===
def train_model():
    model.train()
    for epoch in range(num_epochs):
        total_loss = 0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        avg_loss = total_loss / len(train_loader)
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}")
        evaluate_accuracy(model, val_loader, device)

# === EVALUATE ===
def evaluate_accuracy(model, dataloader, device):
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in dataloader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    acc = 100 * correct / total
    print(f"Validation Accuracy: {acc:.2f}%")
    return acc

# === START TRAINING ===


In [29]:
train_model()

Epoch [1/5], Loss: 1.3835
Validation Accuracy: 68.90%
Epoch [2/5], Loss: 0.9639
Validation Accuracy: 69.30%
Epoch [3/5], Loss: 0.8861
Validation Accuracy: 69.90%
Epoch [4/5], Loss: 0.8409
Validation Accuracy: 70.60%
Epoch [5/5], Loss: 0.8089
Validation Accuracy: 70.75%


In [30]:
evaluate_accuracy(model, train_loader, device)

Validation Accuracy: 74.24%


74.23742374237423